In this file data preprocessing is done on given dataset

In [3]:
import pandas as pd
import json
import os
import datetime
print(os.getcwd())
os.chdir('/content/drive/MyDrive/Task1/Dataset')
print(os.getcwd())

/content/drive/My Drive/Task1
/content/drive/MyDrive/Task1/Dataset


In this step the **csv** file contains **JSON** data so need to sepration of data

In [4]:
import re
from pathlib import Path
p = Path.cwd() / 'zolo-sample-data.csv'
p2 = Path.cwd() / 'emptyData.csv'
with p.open('r') as f:
    with p2.open('w') as f2:
        for cnt, line in enumerate(f):
            if cnt == 0:
                line = line.replace(',', '|')
            else:
                line = re.sub(r',(?=(((?!\}).)*\{)|[^\{\}]*$)', '|', line)
            f2.write(line)

So all the data are combined in one column and delimiter **"|"**

In [6]:
data=pd.read_csv('/content/drive/MyDrive/Task1/Dataset/emptyData.csv')
print(data)

     fields_data|sf_last_modified_at|sf_created_at|website_text|created_at|zolo_lead_id|sf_lead_company_website|sf_lead_company_name|sf_lead_id|sf_lead_name|sf_lead_email|id|user_account_domain
0     {"AnnualRevenue": "", "City": "", "Company": "...                                                                                                                                          
1     {"AnnualRevenue": "", "City": "", "Company": "...                                                                                                                                          
2     {"AnnualRevenue": "", "City": "", "Company": "...                                                                                                                                          
3     {"AnnualRevenue": "", "City": "", "Company": "...                                                                                                                                          
4     {"AnnualRevenue": "", "C

Now data is seprated using **"|"** delimiter and reason is json data is seprated **","** delimiter

In [7]:
df=pd.read_csv('/content/drive/MyDrive/Task1/Dataset/emptyData.csv',converters={'fields_data':json.loads},header=0,error_bad_lines=False,sep="|")
print(df)


b'Skipping line 1882: expected 13 fields, saw 14\nSkipping line 2301: expected 13 fields, saw 14\n'


                                            fields_data  ...  user_account_domain
0     {'AnnualRevenue': '', 'City': '', 'Company': '...  ...             hope.com
1     {'AnnualRevenue': '', 'City': '', 'Company': '...  ...             hope.com
2     {'AnnualRevenue': '', 'City': '', 'Company': '...  ...             hope.com
3     {'AnnualRevenue': '', 'City': '', 'Company': '...  ...             hope.com
4     {'AnnualRevenue': '', 'City': '', 'Company': '...  ...             hope.com
...                                                 ...  ...                  ...
9993  {'AnnualRevenue': '', 'City': '', 'Company': '...  ...  placetechnology.com
9994  {'AnnualRevenue': '', 'City': '', 'Company': '...  ...  placetechnology.com
9995  {'AnnualRevenue': '', 'City': '', 'Company': '...  ...  placetechnology.com
9996  {'AnnualRevenue': '', 'City': '', 'Company': '...  ...  placetechnology.com
9997  {'AnnualRevenue': '', 'City': '', 'Company': '...  ...             hope.com

[9998 rows x 13

Extraction of column of **JSON** data

In [8]:
new_df = df.join(pd.io.json.json_normalize(df["fields_data"]),how='left',lsuffix='_left',rsuffix='_right').drop(["fields_data"],axis=1)
print(new_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


      sf_last_modified_at  ...  job_type__c
0              1597054409  ...          NaN
1              1597126433  ...          NaN
2              1597127006  ...          NaN
3              1597228892  ...          NaN
4              1597299110  ...          NaN
...                   ...  ...          ...
9993           1593528696  ...          NaN
9994           1593528710  ...          NaN
9995           1593528715  ...          NaN
9996           1593528700  ...          NaN
9997           1597394514  ...          NaN

[9998 rows x 333 columns]


Dropping the column that contains **more than 70% null** data

In [9]:
new_df.dropna(axis=1, thresh = int(0.7*new_df.shape[0]), inplace=True)
print(new_df)

      sf_last_modified_at  ...       website_text_right
0              1597054409  ...              offerup.com
1              1597126433  ...              offerup.com
2              1597127006  ...           u-clermont1.fr
3              1597228892  ...                metso.com
4              1597299110  ...               oracle.com
...                   ...  ...                      ...
9993           1593528696  ...            crafted.co.uk
9994           1593528710  ...   balanceinnovations.com
9995           1593528715  ...         iatspayments.com
9996           1593528700  ...      prestwick-group.com
9997           1597394514  ...  physiciancareers.kp.org

[9998 rows x 50 columns]


In [10]:
new_df.isna().sum()

sf_last_modified_at           0
sf_created_at                 0
website_text_left          1981
created_at                    0
sf_lead_company_website    1980
sf_lead_company_name          0
sf_lead_id                    0
sf_lead_name                  0
id                            0
user_account_domain           0
AnnualRevenue                56
City                          0
Company                       0
ConvertedAccountId            0
ConvertedContactId            0
ConvertedDate                 0
ConvertedOpportunityId        0
Country                       0
CreatedById                   0
CreatedDate                   0
Description                2104
Email                         0
EmailBouncedDate              0
EmailBouncedReason            0
FirstName                     0
Id                            0
Industry                      0
IsConverted                   0
IsDeleted                     0
IsUnreadByOwner               0
LastActivityDate              0
LastModi

Dropping the columns which contains **empty string data**

In [11]:
new_df=new_df.drop(['website_text_left','sf_lead_company_website','AnnualRevenue','Description','NumberOfEmployees','Rating'],axis=1)

In [12]:
new_df=new_df.drop(['City','ConvertedAccountId','ConvertedContactId','ConvertedDate','ConvertedOpportunityId','EmailBouncedDate','EmailBouncedReason','Industry','MasterRecordId','Salutation'],axis=1)

Some columns are in **epoch time** so need to convert them to **data and time format**

In [13]:
for i in range(0,9998):
  new_df.sf_created_at[i]=datetime.datetime.fromtimestamp(new_df.sf_created_at[i]).strftime('%c')
  new_df.sf_last_modified_at[i]=datetime.datetime.fromtimestamp(new_df.sf_last_modified_at[i]).strftime('%c')
  new_df.created_at[i]=datetime.datetime.fromtimestamp(new_df.created_at[i]).strftime('%c')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid d

Saving **cleared data** from we doing insights

In [15]:
new_df.to_csv('/content/drive/MyDrive/Task1/Dataset/Dataset.csv')
print("Dataset is ready.......:)")

Dataset is ready.......:)
